# 03 · RuBERT (coarse) + OOD

**Цель.** Дообучить **RuBERT** под coarse-классификацию и внедрить **OOD-детекцию** по CLS-эмбеддингу (L2/z-score), чтобы резать "вне домена".

**Что делаем**

- Fine-tune RuBERT на `train`, валидация на `valid`.
    
- Калибровка softmax-температуры.
    
- OOD: измеряем распределение L2-дистанций CLS к центру/распределению класса; порог по z-score.
    
- Отчеты: метрики по классам, PR/ROC, sanity по hard-кейсам.
    

**Что важно**

- Число классов и их соотношение согласованы с EDA.
    
- Учли длинные тексты на стадии инференса (см. 05: chunk-aware).
    

## Импорты и пути

In [9]:

from pathlib import Path
import json, re, numpy as np, pandas as pd
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.covariance import EmpiricalCovariance

import torch
from torch.utils.data import Dataset

from transformers import (AutoTokenizer, AutoModelForSequenceClassification, AutoModel,
                          Trainer, TrainingArguments, DataCollatorWithPadding)
import evaluate

from inspect import signature

import joblib


DATA_PATH = Path("../data/synthetic_ru_private_ads_50cats_10000_v2.csv")
MODEL_DIR = Path("../data/rubert_cls_model")
MODEL_DIR.mkdir(parents=True, exist_ok=True)

CALIB_PATH  = MODEL_DIR / "temperature.json"
OOD_PATH    = MODEL_DIR / "ood_mahalanobis.joblib"
LABELS_PATH = MODEL_DIR / "label_mapping.json"

BASE_MODEL = "DeepPavlov/rubert-base-cased"


## Данные и очистка

In [2]:

df = pd.read_csv(DATA_PATH)
text_col1, text_col2 = "title", "description"
label_col = "category"

cities = sorted(df["city"].dropna().astype(str).unique().tolist()) if "city" in df.columns else []
phone_re = re.compile(r"(\+?\d[\d\-\s]{6,}\d)")
multi_sp = re.compile(r"\s+")
city_re  = re.compile(r"\b(" + "|".join([re.escape(c) for c in cities]) + r")\b", flags=re.IGNORECASE) if cities else None
noise_re = re.compile(r"\b(продам|куплю|обмен|торг|договор|в наличии)\b", re.IGNORECASE)

def clean_text(s: str) -> str:
    s = str(s)
    s = phone_re.sub(" ", s)
    if city_re is not None:
        s = city_re.sub(" ", s)
    s = noise_re.sub(" ", s)
    s = s.lower()
    s = multi_sp.sub(" ", s).strip()
    return s

df["_text"] = (df[text_col1].fillna("") + " " + df[text_col2].fillna("")).apply(clean_text)
df = df[df["_text"].str.len() > 0].copy()
df = df[["_text", label_col]].rename(columns={"_text": "text", label_col: "label"})
df.head(3)


,text,label
0,продаю автоаксессуары — покажу сегодня. без дт...,Автоаксессуары
1,продаю автоаксессуары — город: . договорная. о...,Автоаксессуары
2,в продаже автоаксессуары — гаражное хранение. ...,Автоаксессуары


## Разбиение + кодировщик меток

In [3]:

le = LabelEncoder()
df["label_id"] = le.fit_transform(df["label"])
labels   = le.classes_.tolist()
n_labels = len(labels)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label_id"])
train_df, val_df  = train_test_split(train_df, test_size=0.1, random_state=42, stratify=train_df["label_id"])

with open(LABELS_PATH, "w", encoding="utf-8") as f:
    json.dump({"labels": labels}, f, ensure_ascii=False, indent=2)

len(train_df), len(val_df), len(test_df), n_labels


(7200, 800, 2000, 50)

## Токенизация и датасеты

In [4]:

tok = AutoTokenizer.from_pretrained(BASE_MODEL)

class AdsDS(Dataset):
    def __init__(self, df, tok):
        self.texts  = df["text"].tolist()
        self.labels = df["label_id"].tolist()
        self.tok = tok
    def __len__(self): return len(self.texts)
    def __getitem__(self, idx):
        enc = self.tok(self.texts[idx], truncation=True, max_length=196)
        enc["labels"] = int(self.labels[idx])
        return enc

ds_train = AdsDS(train_df, tok)
ds_val   = AdsDS(val_df, tok)
ds_test  = AdsDS(test_df, tok)
collator = DataCollatorWithPadding(tok)


In [5]:
import transformers, accelerate
from transformers import TrainingArguments
import inspect
print(transformers.__version__, accelerate.__version__)
print(list(inspect.signature(TrainingArguments.__init__).parameters)[:12])


4.55.4 1.10.1
['self', 'output_dir', 'overwrite_output_dir', 'do_train', 'do_eval', 'do_predict', 'eval_strategy', 'prediction_loss_only', 'per_device_train_batch_size', 'per_device_eval_batch_size', 'per_gpu_train_batch_size', 'per_gpu_eval_batch_size']


## Модель и обучение

In [6]:

id2label = {i: l for i, l in enumerate(labels)}
label2id = {l: i for i, l in enumerate(labels)}

model = AutoModelForSequenceClassification.from_pretrained(
    BASE_MODEL, num_labels=n_labels, id2label=id2label, label2id=label2id
)

metric_acc = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, y = eval_pred
    preds = logits.argmax(-1)
    return {
        "accuracy": metric_acc.compute(predictions=preds, references=y)["accuracy"],
        "macro_f1": f1_score(y, preds, average="macro"),
    }

def make_training_args(**overrides):
    base = dict(
        output_dir=str(MODEL_DIR / "runs"),
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=32,
        num_train_epochs=3,
        weight_decay=0.01,
        logging_steps=50,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        metric_for_best_model="macro_f1",
        load_best_model_at_end=True,
        report_to="none",
    )
    base.update(overrides)
    sig = set(signature(TrainingArguments.__init__).parameters)
    if "evaluation_strategy" not in sig and "eval_strategy" in sig:
        base["eval_strategy"] = base.pop("evaluation_strategy")
    safe = {k: v for k, v in base.items() if k in sig}
    return TrainingArguments(**safe)

args = make_training_args()

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=ds_train,
    eval_dataset=ds_val,
    tokenizer=tok,
    data_collator=collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate(ds_test)
trainer.save_model(str(MODEL_DIR))
tok.save_pretrained(str(MODEL_DIR))




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at DeepPavlov/rubert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/var/folders/jb/lps7hzhd0_s0kg1l674s2htc0000gn/T/ipykernel_34804/635054018.py:84: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/Users/maxim/Documents/GitHub/report-analysis/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Macro F1
1,0.323200,0.206575,1.000000,1.000000
2,0.056700,0.040578,1.000000,1.000000
3,0.040000,0.029827,1.000000,1.000000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/Users/maxim/Documents/GitHub/report-analysis/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/maxim/Documents/GitHub/report-analysis/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/maxim/Documents/GitHub/report-analysis/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argumen

('../data/rubert_cls_model/tokenizer_config.json',
 '../data/rubert_cls_model/special_tokens_map.json',
 '../data/rubert_cls_model/vocab.txt',
 '../data/rubert_cls_model/added_tokens.json',
 '../data/rubert_cls_model/tokenizer.json')

## Температурная калибровка

In [7]:

import numpy as np, json

val_logits = trainer.predict(ds_val).predictions
val_y = val_df["label_id"].to_numpy()

def nll_with_T(T, logits, y):
    z = logits / T
    z = z - z.max(axis=1, keepdims=True)
    p = np.exp(z); p /= (p.sum(axis=1, keepdims=True) + 1e-9)
    return float(-np.log(p[np.arange(len(y)), y] + 1e-12).mean())

T = 1.0
for _ in range(200):
    cand = np.linspace(max(0.5, T-0.5), T+0.5, 11)
    vals = [nll_with_T(c, val_logits, val_y) for c in cand]
    j = int(np.argmin(vals))
    if abs(cand[j] - T) < 1e-3:
        T = float(cand[j]); break
    T = float(cand[j])

with open(CALIB_PATH, "w") as f:
    json.dump({"temperature": T}, f)

T


/Users/maxim/Documents/GitHub/report-analysis/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


0.5

## OOD: Mahalanobis по CLS‑эмбеддингам (+порог)

In [26]:

from sklearn.covariance import LedoitWolf
from transformers import AutoModel, AutoTokenizer
import numpy as np, joblib, json, torch

# fallback, если clean_text не объявлен
try:
    clean_text
except NameError:
    clean_text = lambda s: s

tok = AutoTokenizer.from_pretrained(str(MODEL_DIR))
backbone = AutoModel.from_pretrained(str(MODEL_DIR)); backbone.eval()

def cls_embeddings(texts, batch=64):
    out = []
    with torch.no_grad():
        for i in range(0, len(texts), batch):
            enc = tok(texts[i:i+batch], truncation=True, max_length=196,
                      return_tensors="pt", padding=True)
            cls = backbone(**enc).last_hidden_state[:, 0, :].cpu().numpy()
            out.append(cls)
    return np.vstack(out).astype("float64")

train_texts = [clean_text(t) for t in train_df["text"].tolist()]
val_texts   = [clean_text(t) for t in val_df["text"].tolist()]

train_embs = cls_embeddings(train_texts)
val_embs   = cls_embeddings(val_texts)

ytr   = train_df["label_id"].to_numpy()
H     = train_embs.shape[1]
means = np.vstack([train_embs[ytr == c].mean(axis=0) for c in range(len(labels))])  # [C,H]

resid = train_embs - means[ytr]
lw = LedoitWolf().fit(resid)
P  = lw.get_precision().astype("float64")  # [H,H]

d2_train = np.einsum("ij,jk,ik->i", resid, P, resid)
scale = H / float(d2_train.mean())
P *= scale  # нормировка масштаба

diff_val = val_embs[:, None, :] - means[None, :, :]
d2_val   = np.einsum("nch,hk,nck->nc", diff_val, P, diff_val)
dmin_val = d2_val.min(axis=1)

thr_q = float(np.quantile(dmin_val, 0.995))
mu    = float(dmin_val.mean())
sigma = float(dmin_val.std(ddof=1))

alpha = 1.10
z_thr = 4.0
ood_conf_gate = 0.75  # применять OOD только при низкой уверенности

joblib.dump({"means": means.astype("float32"), "precision": P}, OOD_PATH)
with open(MODEL_DIR / "ood_threshold.json", "w") as f:
    json.dump({
        "threshold": thr_q, "mu": mu, "sigma": sigma,
        "alpha": alpha, "z_thr": z_thr, "ood_conf_gate": ood_conf_gate
    }, f)

print({
    "H": H,
    "mean_d2_train_after_scale": float(np.einsum("ij,jk,ik->i", resid, P, resid).mean()),
    "thr_q": thr_q, "mu": mu, "sigma": sigma
})





{'H': 768, 'mean_d2_train_after_scale': 768.0, 'thr_q': 2803.175209906024, 'mu': 860.5635329225602, 'sigma': 414.4806863786894}


## Инференс + виджет

In [30]:

import json, joblib, numpy as np, ipywidgets as W, pandas as pd, torch
from transformers import AutoModel, AutoTokenizer
from IPython.display import display, HTML, clear_output

with open(CALIB_PATH) as f: T = float(json.load(f)["temperature"])
with open(MODEL_DIR / "label_mapping.json") as f: labels = json.load(f)["labels"]

ood_meta = json.load(open(MODEL_DIR / "ood_threshold.json"))
OOD_THR = float(ood_meta["threshold"])
MU      = float(ood_meta["mu"])
SIGMA   = float(ood_meta["sigma"])
ALPHA   = float(ood_meta.get("alpha", 1.10))
Z_THR   = float(ood_meta.get("z_thr", 4.0))
OOD_CONF_GATE = float(ood_meta.get("ood_conf_gate", 0.75))

mm = joblib.load(OOD_PATH); means, P = mm["means"].astype("float64"), mm["precision"].astype("float64")

tok = AutoTokenizer.from_pretrained(str(MODEL_DIR))
backbone = AutoModel.from_pretrained(str(MODEL_DIR)); backbone.eval()

try: trainer.args.dataloader_pin_memory = False
except: pass

# fallback, если clean_text не объявлен
try:
    clean_text
except NameError:
    clean_text = lambda s: s

def predict_texts(texts, tau_other=0.35, use_ood=True, topk=5, ood_conf_gate=OOD_CONF_GATE):
    texts_norm = [clean_text(t) for t in texts]

    tmp_ds = AdsDS(pd.DataFrame({"text": texts_norm, "label_id": [0]*len(texts_norm)}), tok)
    logits = trainer.predict(tmp_ds).predictions

    z = logits / T
    z -= z.max(axis=1, keepdims=True)
    p = np.exp(z); p /= (p.sum(axis=1, keepdims=True) + 1e-9)
    p_max = p.max(axis=1)

    flags = [False] * len(texts_norm)
    if use_ood:
        with torch.no_grad():
            enc = tok(texts_norm, truncation=True, max_length=196, return_tensors="pt", padding=True)
            v = backbone(**enc).last_hidden_state[:, 0, :].cpu().numpy().astype("float64")
        diff = v[:, None, :] - means[None, :, :]
        d2   = np.einsum("bch,hk,bck->bc", diff, P, diff)
        dmin = d2.min(axis=1)
        zscr = (dmin - MU) / (SIGMA + 1e-9)
        raw_ood = (dmin > OOD_THR * ALPHA) & (zscr > Z_THR)
        flags   = (raw_ood & (p_max < float(ood_conf_gate))).tolist()

    res = []
    for i in range(len(texts_norm)):
        order = np.argsort(-p[i])[:topk]
        pred = labels[int(order[0])]; conf = float(p[i, order[0]])
        pred_out = pred if (not flags[i] and conf >= float(tau_other)) else "Other"
        res.append({
            "text": texts[i],
            "pred": pred_out,
            "confidence": conf,
            "topk_labels": [labels[int(j)] for j in order],
            "topk_scores": [float(p[i, j]) for j in order],
            "ood": bool(flags[i]),
        })
    return res


# sanity-check: список для проверки RuBERT
samples = [
    # Инструменты / Ремонт
    "Продам валик для покраски стен в доме",
    "Шлифовка стен, укладка плитки и стяжка пола — работаем по договору",
    "Куплю перфоратор б/у, срочно",

    # Авто
    "Продаю запчасти для авто, оригинал OEM, подходят на Kia/Hyundai",
    "Продаю автомобиль 2022 года без пробега, АКПП, ПТС на руках",
    "Шины зимние, радиус 17, состояние отличное",

    # Электроника
    "Продам ноутбук Lenovo i5 8GB 256GB SSD, состояние отличное",
    "Смартфон Samsung Galaxy S22, полный комплект",
    "Планшет Apple iPad Pro 2021, 11 дюймов",

    # Недвижимость
    "Квартира в Москве, 2 комнаты, продажа от собственника",
    "Сдаю комнату в Санкт-Петербурге на длительный срок",
    "Дом с земельным участком 15 соток в Твери",

    # Бытовая техника
    "Стиральная машина Bosch, загрузка 6 кг",
    "Пылесос Dyson, беспроводной, в отличном состоянии",
    "Кондиционер Mitsubishi, установка в подарок",

    # Разное
    "Есть в наличии книги, переплет твердый, издательство АСТ",
    "Товары для мам и малышей, коляска 3 в 1",
    "Отдам даром диван б/у, самовывоз",
    "Услуги по составлению резюме"
]

df_test = pd.DataFrame(predict_texts(samples, tau_other=0.35, use_ood=True))
pd.set_option("display.max_colwidth", 80)
df_test[["text","pred","confidence","ood","topk_labels","topk_scores"]]



,text,pred,confidence,ood,topk_labels,topk_scores
0,Продам валик для покраски стен в доме,Other,0.342149,True,"[Комнаты — аренда, Услуги ремонта, Мебель, Квартиры — продажа, Микроволновые...","[0.3421489894390106, 0.13933920860290527, 0.10741438716650009, 0.07248795032..."
1,"Шлифовка стен, укладка плитки и стяжка пола — работаем по договору",Other,0.303121,True,"[Ремонт и стройматериалы, Услуги ремонта, Мебель, Микроволновые печи, Комнат...","[0.3031211197376251, 0.22545139491558075, 0.12447251379489899, 0.07714072614..."
2,"Куплю перфоратор б/у, срочно",Other,0.355694,True,"[Фотоаппараты, Комнаты — аренда, Украшения и часы, Телевизоры и видео, Услуг...","[0.3556944727897644, 0.11376142501831055, 0.10871344059705734, 0.04769259691..."
3,"Продаю запчасти для авто, оригинал OEM, подходят на Kia/Hyundai",Запчасти для авто,0.999199,False,"[Запчасти для авто, Товары для мам и малышей, Мотоциклы и мототехника, Грузо...","[0.9991992115974426, 9.240340295946226e-05, 5.601883458439261e-05, 5.3935862..."
4,"Продаю автомобиль 2022 года без пробега, АКПП, ПТС на руках",Легковые автомобили,0.947762,False,"[Легковые автомобили, Запчасти для авто, Автоаксессуары, Стиральные машины, ...","[0.9477624893188477, 0.015432487241923809, 0.007108770776540041, 0.005793243..."
5,"Шины зимние, радиус 17, состояние отличное",Шины и диски,0.998010,False,"[Шины и диски, Водный транспорт, Пылесосы, Книги, Холодильники]","[0.9980098605155945, 0.00020002831297460943, 0.00012117702863179147, 0.00011..."
6,"Продам ноутбук Lenovo i5 8GB 256GB SSD, состояние отличное",Ноутбуки,0.990231,False,"[Ноутбуки, Планшеты, Смартфоны, Автоаксессуары, Стиральные машины]","[0.9902308583259583, 0.0043939086608588696, 0.001046575140208006, 0.00065118..."
7,"Смартфон Samsung Galaxy S22, полный комплект",Смартфоны,0.997019,False,"[Смартфоны, Автоаксессуары, Квартиры — аренда, Планшеты, Ноутбуки]","[0.997019350528717, 0.00036557306884787977, 0.00035705635673366487, 0.000259..."
8,"Планшет Apple iPad Pro 2021, 11 дюймов",Планшеты,0.914068,False,"[Планшеты, Смартфоны, Ноутбуки, Автоаксессуары, Стиральные машины]","[0.914068341255188, 0.0523296594619751, 0.019579263404011726, 0.001642570598..."
9,"Квартира в Москве, 2 комнаты, продажа от собственника",Other,0.428677,True,"[Квартиры — продажа, Квартиры — аренда, Комнаты — аренда, Дома и дачи — арен...","[0.4286769926548004, 0.2880663573741913, 0.24081367254257202, 0.014792103320..."


In [28]:
samples = [
    "Продам валик для покраски стен в доме",
    "Продаю запчасти для авто, оригинал OEM, подходят на Kia/Hyundai",
    "Продам ноутбук Lenovo i5 8GB 256GB SSD, состояние отличное",
]

with torch.no_grad():
    tn = [clean_text(t) for t in samples]
    enc = tok(tn, truncation=True, max_length=196, return_tensors="pt", padding=True)
    v = backbone(**enc).last_hidden_state[:,0,:].cpu().numpy().astype("float64")
diff = v[:, None, :] - means[None, :, :]
d2   = np.einsum("bch,hk,bck->bc", diff, P, diff)
dmin = d2.min(axis=1)
print("dmin:", dmin, "thr:", OOD_THR, "alpha:", ALPHA, "mu:", MU, "sigma:", SIGMA, "z:", (dmin-MU)/SIGMA)


dmin: [337690.11431136   3982.6162989   97846.21301216] thr: 2803.175209906024 alpha: 1.1 mu: 860.5635329225602 sigma: 414.4806863786894 z: [812.65439343   7.5324445  233.99316944]


## Итоги (03 · RuBERT + OOD)

- Обучен основной coarse-классификатор на RuBERT.
    
- Добавлен OOD-блок (z-score по CLS-L2), что снижает шум "вне домена".
    
- Подготовлена база для саб-классификаторов и последующей калибровки порогов.
    

**Дальше:**  
(1) применить тонкие саб-головы по фиксированным эмбеддингам (см. 04),  
(2) подобрать глобальный `τ_other` и `temperature` (см. 06),  
(3) для прод-инференса включить chunk-aware (см. 05).